# (header)

## GPIO

### GPIO OUTPUT to LED

In [ ]:
from RPi import GPIO
#Pin numbering LED01 Pin 13 (GPIO 27) ; LED02 Pin 15 (GPIO 22)
GPIO.setmode(GPIO.BOARD)
LED01 = 13
LED02 = 15
#To input(read) 
GPIO.setup(LED01,GPIO.OUT) ##LED01 
GPIO.setup(LED02,GPIO.OUT) ##LED02

In [ ]:
GPIO.output(LED01,GPIO.HIGH) #state can be 0 / GPIO.LOW / False or 1 / GPIO.HIGH / True.
GPIO.output(LED02,GPIO.HIGH)

#### blanking LED01

In [ ]:
from time import sleep
loops = 10
for i in range(loops):
    GPIO.output(LED01,GPIO.HIGH)
    sleep(0.3)
    GPIO.output(LED01,GPIO.LOW)
    sleep(0.3)

#### abstracting LED ON/OFF

In [ ]:
import RPi.GPIO as GPIO

from time import sleep
GPIO.cleanup() 
GPIO.setmode(GPIO.BOARD)

class RPI_LED:
    def __init__(self,pin):
        self.pin = pin
        GPIO.setup(pin,GPIO.OUT)
        self.off()
    def on(self):
         GPIO.output(self.pin,GPIO.HIGH)
    def off(self):
         GPIO.output(self.pin,GPIO.LOW)
    def blinking(self):
        self.on()
        sleep(0.4)
        self.off()
            

        

   

In [ ]:
led_01 = RPI_LED(13)
led_01.blinking()
led_02 = RPI_LED(15)
led_02.blinking()

In [ ]:
import threading
from threading import Thread
loops = 20
for i in range(loops):
    Thread(target=led_01.blinking()).start()
    Thread(target=led_02.blinking()).start()

In [ ]:
import threading
from threading import Thread
loops = 5
for i in range(loops):
    Thread(target=led_01.on()).start()
    Thread(target=led_02.on()).start()
    sleep(0.4)
    Thread(target=led_01.off()).start()
    Thread(target=led_02.off()).start()
    sleep(0.4)
    

In [3]:
import RPi.GPIO as GPIO
from time import sleep
GPIO.cleanup()
pins = [13,15]
GPIO.setmode(GPIO.BOARD)
GPIO.setup(pins,GPIO.OUT)
led_01 = GPIO.PWM(pins[0], 100)
led_02 = GPIO.PWM(pins[1], 100)
led_01.start(100)
led_02.start(0)
for i in range(2):
    for i in range(0,101):      # 101 because it stops when it finishes 100
            led_01.ChangeDutyCycle(100-i)  
            led_02.ChangeDutyCycle(i)  
            sleep(0.03)  
    for i in range(100,-1,-1):
            led_01.ChangeDutyCycle(100-i)   
            led_02.ChangeDutyCycle(i)
            sleep(0.03)
led_01.stop()
led_02.stop()

### GPIO input to read buttons

In [96]:
import RPi.GPIO as GPIO

#Pin numbering
## Buttons鍵 SW01 Pin 16 (GPIO23) ; SW02 Pin 18 (GPIO24)
GPIO.setmode(GPIO.BCM)
#To input(read) 
GPIO.setup(23,GPIO.IN,pull_up_down = GPIO.PUD_DOWN) ##left sw01 
GPIO.setup(24,GPIO.IN) ##right sw02

### POLLING

In [ ]:
# read SW BUTTON
GPIO.input(23) # 0 is pressed 1 is unaaapressed

In [6]:
if (GPIO.input(23) == GPIO.HIGH):
    print('Input was not pressed')
else:
    print('Input was pressed')

Input was not pressed


### INTERRUPT and CALLBACK


In [2]:
# Wait until the SW button is pressed
import RPi.GPIO as GPIO
GPIO.cleanup()
GPIO.setmode(GPIO.BCM)
GPIO.setup(23,GPIO.IN,pull_up_down = GPIO.PUD_DOWN) ##left sw01 
GPIO.wait_for_edge(23, GPIO.RISING) #block python execution of your program until an edge is detected - 

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:2: RuntimeWarning: No channels have been set up yet - nothing to clean up!  Try cleaning up at the end of your program instead!
  from ipykernel import kernelapp as app


23

In [12]:
# Wait in a timly fashion

import RPi.GPIO as GPIO
GPIO.cleanup()
GPIO.setmode(GPIO.BCM)
GPIO.setup(23,GPIO.IN,pull_up_down = GPIO.PUD_DOWN) ##left sw01 
channel = GPIO.wait_for_edge(23, GPIO.RISING, timeout=5000)
if channel is None:
    print('5 SECS timeout occurred')
else:
    print('Edge detected on channel', channel)

5 SECS timeout occurred


In [13]:
# did event happen - was SW01 ever pressed?  

import RPi.GPIO as GPIO
from time import sleep

def print_some_nums():
    for i in range(11):
        print("i:",i)
        sleep(0.5)

GPIO.cleanup()
GPIO.setmode(GPIO.BCM)
GPIO.setup(23,GPIO.IN,pull_up_down = GPIO.PUD_DOWN) ##left sw01 
GPIO.add_event_detect(23, GPIO.RISING)  # add rising edge detection on a channel
print_some_nums()
if GPIO.event_detected(23):
    print('Button pressed')

i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10


In [76]:
def callbackFun_one(channel):
    print('Edge detected on channel %s'%channel)
    print('Callback func one')

def callbackFun_two(channel):
    print('Edge detected on channel %s'%channel)
    print('Callback func two')

In [84]:
GPIO.remove_event_detect(23)
GPIO.cleanup()
GPIO.setmode(GPIO.BCM)
GPIO.setup(23,GPIO.IN) ##left sw01 
GPIO.add_event_detect(23, GPIO.RISING)
GPIO.add_event_callback(23, callbackFun_one)

Edge detected on channel 23
Callback func one
Edge detected on channel 23
Callback func one
Edge detected on channel 23
Callback func one
Edge detected on channel 23
Callback func one
Edge detected on channel 23
Callback func one
Edge detected on channel 23
Callback func one


In [94]:
GPIO.remove_event_detect(23)
GPIO.remove_event_detect(24)


### Switch debounce

You may notice that the callbacks are called more than once for each button press. This is as a result of what is known as 'switch bounce'. There are two ways of dealing with switch bounce:
add a 0.1uF capacitor across your switch.
software debouncing
a combination of both
To debounce using software, add the bouncetime= parameter to a function where you specify a callback function. Bouncetime should be specified in milliseconds. 

In [95]:

GPIO.cleanup()
GPIO.setmode(GPIO.BCM)
GPIO.setup(23,GPIO.IN) ##left sw01 
GPIO.setup(24,GPIO.IN) ##left sw02
GPIO.add_event_detect(23, GPIO.RISING, callback=callbackFun_one, bouncetime=50)
GPIO.add_event_detect(24, GPIO.RISING, callback=callbackFun_two)

Edge detected on channel 23
Callback func one
Edge detected on channel 23
Callback func one
Edge detected on channel 24
Callback func two
Edge detected on channel 24
Callback func two
Edge detected on channel 24
Callback func two
Edge detected on channel 24
Callback func two
Edge detected on channel 23
Callback func one
Edge detected on channel 24
Callback func two
Edge detected on channel 24
Callback func two
Edge detected on channel 24
Callback func two
Edge detected on channel 24
Callback func two
